In [1]:
import pandas as pd
import numpy as np

path_to_text = 'spam.csv'
data = pd.read_csv(path_to_text, encoding='latin-1')[['v1', 'v2']]
text = data['v2']
label = data['v1']

data[10:14]

,v1,v2
10,ham,I'm gonna be home soon and i don't want to tal...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
12,spam,URGENT! You have won a 1 week FREE membership ...
13,ham,I've been searching for the right words to tha...


In [2]:
data[5:6]

,v1,v2
5,spam,FreeMsg Hey there darling it's been 3 week's n...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = list(text)
tfidf = TfidfVectorizer(max_features = 6000) 
tfidf.fit(corpus)
tfidf_features = tfidf.transform(corpus)

In [4]:
print(tfidf_features.shape)

(5572, 6000)


In [5]:
from sklearn.svm import SVC
train_percent = 0.7
train_cutoff = int(np.floor(train_percent*len(text) ) )
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(tfidf_features[0 : train_cutoff], 
                  label[0 : train_cutoff])
svm_predictions = svm_model_linear.predict(
                  tfidf_features[train_cutoff + 1 : len(text)])

In [6]:
svm_predictions

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [15]:
from sklearn.metrics import precision_recall_fscore_support

In [20]:

results = pd.DataFrame(index = ['TF-IDF'], 
          columns = ['Precision', 'Recall', 'F1 score', 'support']
          )
results.loc['TF-IDF'] = precision_recall_fscore_support(
          label[train_cutoff + 1 : len(text)], 
          svm_predictions, 
          average = 'binary'
          )

ValueError: pos_label=1 is not a valid label: array(['ham', 'spam'], dtype='<U4')

In [22]:
 svm_predictions[5]


'ham'

In [ ]:
results